In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

train = pd.read_csv("train.csv", parse_dates=['date'], dayfirst=True)
test = pd.read_csv("test.csv", parse_dates=['date'], dayfirst=True)

def extract_date(df, column):
    df[column+'_year'] = df[column].apply(lambda x: x.year)
    df[column+'_month'] = df[column].apply(lambda x: x.month)
    df[column+'_day'] = df[column].apply(lambda x: x.day)
    df[column+'_weekday'] = df[column].apply(lambda x: x.weekday())
    df[column+'_hour'] = df[column].apply(lambda x: x.hour)
    
extract_date(train, 'date')
extract_date(test, 'date')

X_train = train.loc[:, ['date_year', 'date_month', 'date_day', 'date_weekday', 'date_hour']].values
y_train = train.loc[:, ['speed']].values
X_test = test.loc[:, ['date_year', 'date_month', 'date_day', 'date_weekday', 'date_hour']].values

# y_test = y_test.ravel()
y_train = y_train.ravel()

est = HistGradientBoostingRegressor(max_iter=1000,
                                    max_depth=16,
                                    random_state=100,
                                    min_samples_leaf=2,
                                    learning_rate=0.1).fit(X_train, y_train)
result = est.predict(X_train)

print('Train:',mean_squared_error(y_train,result))

submit = est.predict(X_test)

test['speed'] = submit
submit = test[['id', 'speed']]

#submit.to_csv('./histreg.csv', index=False)

Train: 7.555486314470185
